In [1]:
import pandas as pd
import re
from pyhanlp import *
import json

import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
data = []
label = []
for root, dirs, files in os.walk('./分词结果/'): 
    for file in files:
        if '其它' not in file:
            fr = open('./分词结果/' + file, 'r',encoding='utf-8')
            for line in fr.readlines():
                line = [word.split('/')[0] for word in line.split(' ') if 'n' in word]
                data.append(line)
                label.append(file.split('.')[0])

In [23]:
dataf = pd.DataFrame({'类别':label,'内容':data})

In [18]:
df.类别.unique()

array(['婚恋交友', '电子电器', '网络服务', '旅游交通', '教育培训', '网络购物', '其它', '共享服务',
       '房产家居', '金融保险', '汽车行业'], dtype=object)

In [24]:
dataf.类别.unique()

array(['婚恋交友', '电子电器', '网络服务', '旅游交通', '教育培训', '网络购物', '共享服务', '房产家居',
       '金融保险', '汽车行业'], dtype=object)

In [26]:
import nltk
train, test = train_test_split(dataf, test_size=0.3, random_state=46)
train_tagged = train.apply(
    lambda r: TaggedDocument(words=r.内容, tags=[r.类别]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=r.内容, tags=[r.类别]), axis=1)

In [33]:
train_tagged.values[4]


TaggedDocument(words=['小象', '优品', '笔', '借款', '平台', '贷款', '负债', '失业', '资金周转', '困难', '小象', '优品', '协商', '本金', '客服', '生命', '人', '笔', '借款', '砍头', '息', '月', '利息', '本金', '上岸', '找朋友', '本金', '钱', '协商', '小象', '优品', '平台', '账单', '借款', '合同', '砍头', '息', '小象', '优品', '可能会', '轻生', '本金', '银行贷款', '相符合', '利息', '心理', '压力'], tags=['金融保险'])

In [34]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [35]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 692322/692322 [00:00<00:00, 3348920.68it/s]


In [36]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 692322/692322 [00:00<00:00, 3242012.43it/s]


CPU times: user 30min 55s, sys: 4min 19s, total: 35min 15s
Wall time: 20min 32s


In [37]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [38]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.5226517474975565
Testing F1 score: 0.39626688830422335


/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
